In [7]:
import atexit
import base64
import dash
from dash import html, dcc, Input, Output, State
import mysql.connector
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Load real estate data
real_estate_data = pd.read_csv("Real_Estate.csv")  # Replace the path with your path

# Define features and target
features = ['distance_to_mrt', 'stores', 'latitude', 'longitude']
target = 'house_price_of_unit_area'
X = real_estate_data[features]
y = real_estate_data[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Database setup
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='house'
)
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS real_estate_predictions (
        id INT AUTO_INCREMENT PRIMARY KEY,
        distance_to_mrt DOUBLE,
        stores DOUBLE,
        latitude DOUBLE,
        longitude DOUBLE,
        prediction DOUBLE,
        prediction_score DOUBLE
    )
''')
conn.commit()

# Register function to close the database connection on exit
atexit.register(lambda: (cursor.close() if cursor else None, conn.close() if conn else None))

# Dash app setup
app = dash.Dash(__name__)

# Encode background image to base64
image_filename = 'OIG.jpeg'  # Replace with the path to your image file
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div([
        html.H1("Real Estate Price Prediction", style={'text-align': 'center'}),
        html.Div([
            dcc.Input(id='distance_to_mrt', type='number', placeholder='Distance to MRT Station (meters)',
                      style={'margin': '10px', 'padding': '10px'}),
            dcc.Input(id='stores', type='number', placeholder='Number of Convenience Stores',
                      style={'margin': '10px', 'padding': '10px'}),
            dcc.Input(id='latitude', type='number', placeholder='Latitude',
                      style={'margin': '10px', 'padding': '10px'}),
            dcc.Input(id='longitude', type='number', placeholder='Longitude',
                      style={'margin': '10px', 'padding': '10px'}),
            html.Button('Predict Price', id='predict_button', n_clicks=0,
                        style={'margin': '10px', 'padding': '10px', 'background-color': '#007BFF', 'color': 'white'}),
        ], style={'text-align': 'center'}),
        html.Div(id='prediction_output', style={'text-align': 'center', 'font-size': '20px', 'margin-top': '20px'}),
    ], style={'width': '50%', 'margin': '0 auto', 'border': '2px solid #007BFF', 'padding': '20px',
              'border-radius': '10px', 'background-image': f'url("data:image/png;base64,{encoded_image.decode()}")'})
])

def predict_house_price(distance_to_mrt, stores, latitude, longitude):
    # Prepare the feature vector
    features = pd.DataFrame([[distance_to_mrt, stores, latitude, longitude]],
                            columns=['distance_to_mrt', 'stores', 'latitude', 'longitude'])

    # Predict
    prediction = model.predict(features)[0]

    # Prediction score
    prediction_score = model.score(X_test, y_test)

    return prediction, prediction_score

def save_prediction_to_database(distance_to_mrt, stores, latitude, longitude, prediction, prediction_score):
    cursor.execute('''
        INSERT INTO real_estate_predictions (distance_to_mrt, stores, latitude, longitude, prediction, prediction_score)
        VALUES (%s, %s, %s, %s, %s, %s)
    ''', (distance_to_mrt, stores, latitude, longitude, prediction, prediction_score))
    conn.commit()

@app.callback(
    Output('prediction_output', 'children'),
    [Input('predict_button', 'n_clicks')],
    [
        State('distance_to_mrt', 'value'),
        State('stores', 'value'),
        State('latitude', 'value'),
        State('longitude', 'value')
    ]
)
def update_output(n_clicks, distance_to_mrt, stores, latitude, longitude):
    if n_clicks > 0 and all(v is not None for v in [distance_to_mrt, stores, latitude, longitude]):
        prediction, prediction_score = predict_house_price(distance_to_mrt, stores, latitude, longitude)

        # Store input values and prediction score in the database
        save_prediction_to_database(distance_to_mrt, stores, latitude, longitude, prediction, prediction_score)

        return f'Predicted House Price of Unit Area: {prediction:.2f}, Prediction Score: {prediction_score:.4f}'
    elif n_clicks > 0:
        return 'Please enter all values to get a prediction'
    return ''

if __name__ == '__main__':
    app.run_server(port=8058)
